In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [22]:
url = r"https://www.goodreads.com/author/quotes/656983.J_R_R_Tolkien?page=1"

driver = webdriver.Chrome()
driver.get(url)